In [0]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime

In [2]:
from google.colab import files
uploaded = files.upload()

Saving doc_reports.csv to doc_reports.csv


In [3]:
uploaded = files.upload()

Saving facial_similarity_reports.csv to facial_similarity_reports.csv


In [0]:
doc_reports = pd.read_csv('doc_reports.csv',index_col = 0)
facial_reports = pd.read_csv('facial_similarity_reports.csv',index_col=0)

In [0]:
x=doc_reports['properties'].map(eval)
y=x.apply(pd.Series)

In [0]:
doc_reports=pd.concat([doc_reports.drop(['properties'], axis=1), y], axis=1)

In [0]:
doc_reports['created_at'] = pd.to_datetime(doc_reports['created_at'])
doc_reports['date_of_expiry'] = pd.to_datetime(doc_reports['date_of_expiry'],errors='coerce')
doc_reports['created_at'] = doc_reports['created_at'].dt.strftime('%Y-%m-%d')

In [0]:
facial_reports['created_at'] = pd.to_datetime(facial_reports['created_at'])
facial_reports['created_at'] = facial_reports['created_at'].dt.strftime('%Y-%m-%d')

In [18]:
doc_reports['user_id'].equals(facial_reports['user_id'])

True

In [0]:
df = facial_reports.merge(doc_reports, on = facial_reports.index,)
df = df.drop(['key_0'], 1)

In [20]:
df.head()

,user_id_x,result_x,face_comparison_result,created_at_x,facial_image_integrity_result,visual_authenticity_result_x,properties,attempt_id_x,user_id_y,result_y,...,compromised_document_result,sub_result,gender,nationality,document_type,date_of_expiry,issuing_country,issuing_date,issuing_state,document_version
0,ab23fae164e34af0a1ad1423ce9fd9f0,clear,clear,2017-06-20,clear,consider,{},050a0596de424fab83c433eaa18b3f8d,ab23fae164e34af0a1ad1423ce9fd9f0,consider,...,NaN,caution,Male,IRL,passport,2019-08-12,IRL,NaN,NaN,NaN
1,15a84e8951254011b47412fa4e8f65b8,clear,clear,2017-06-20,clear,clear,{},f69c1e5f45a64e50a26740b9bfb978b7,15a84e8951254011b47412fa4e8f65b8,clear,...,NaN,clear,Female,NaN,driving_licence,2023-02-28,GBR,NaN,NaN,NaN
2,ffb82fda52b041e4b9af9cb4ef298c85,clear,clear,2017-06-20,clear,clear,{},f9f84f3055714d8e8f7419dc984d1769,ffb82fda52b041e4b9af9cb4ef298c85,clear,...,NaN,clear,Male,ITA,passport,2018-06-09,ITA,NaN,NaN,NaN
3,bd4a8b3e3601427e88aa1d9eab9f4290,clear,clear,2017-06-20,clear,clear,{},10a54a1ecf794404be959e030f11fef6,bd4a8b3e3601427e88aa1d9eab9f4290,clear,...,NaN,clear,Male,NaN,national_identity_card,NaT,FRA,2007-08,NaN,NaN
4,f52ad1c7e69543a9940c3e7f8ed28a39,clear,clear,2017-06-20,clear,clear,{},1f320d1d07de493292b7e0d5ebfb1cb9,f52ad1c7e69543a9940c3e7f8ed28a39,clear,...,NaN,clear,Male,POL,national_identity_card,2019-05-29,POL,NaN,NaN,NaN


In [0]:
duplicated = set()
for column in df.columns:
    if "_x" in column or "_y" in column:
        name = column.replace("_x", "")
        name = name.replace("_y", "")
        duplicated.add(name)


def compare_columns(column, df):
    return df[column + "_x"].equals(df[column + "_y"])

In [0]:
for name in duplicated:
    facial_col = name + "_x"
    docs_col = name + "_y"
    if compare_columns(name, df):
        df.rename({facial_col: name}, axis=1, inplace=True)
        df.drop(docs_col, inplace=True, axis=1)
    else:
        df.rename({facial_col: "Facial_" + name, docs_col: "Docs_" +
                   name}, axis=1, inplace=True)

In [26]:
df.head()

,user_id,FACIAL_result,face_comparison_result,created_at,facial_image_integrity_result,visual_authenticity_result,properties,FACIAL_attempt_id,DOCS_result,image_integrity_result,...,compromised_document_result,sub_result,gender,nationality,document_type,date_of_expiry,issuing_country,issuing_date,issuing_state,document_version
0,ab23fae164e34af0a1ad1423ce9fd9f0,clear,clear,2017-06-20,clear,consider,{},050a0596de424fab83c433eaa18b3f8d,consider,clear,...,NaN,caution,Male,IRL,passport,2019-08-12,IRL,NaN,NaN,NaN
1,15a84e8951254011b47412fa4e8f65b8,clear,clear,2017-06-20,clear,clear,{},f69c1e5f45a64e50a26740b9bfb978b7,clear,clear,...,NaN,clear,Female,NaN,driving_licence,2023-02-28,GBR,NaN,NaN,NaN
2,ffb82fda52b041e4b9af9cb4ef298c85,clear,clear,2017-06-20,clear,clear,{},f9f84f3055714d8e8f7419dc984d1769,clear,clear,...,NaN,clear,Male,ITA,passport,2018-06-09,ITA,NaN,NaN,NaN
3,bd4a8b3e3601427e88aa1d9eab9f4290,clear,clear,2017-06-20,clear,clear,{},10a54a1ecf794404be959e030f11fef6,clear,clear,...,NaN,clear,Male,NaN,national_identity_card,NaT,FRA,2007-08,NaN,NaN
4,f52ad1c7e69543a9940c3e7f8ed28a39,clear,clear,2017-06-20,clear,clear,{},1f320d1d07de493292b7e0d5ebfb1cb9,clear,clear,...,NaN,clear,Male,POL,national_identity_card,2019-05-29,POL,NaN,NaN,NaN


In [0]:
df.sort_values(["created_at"], inplace=True)
df.reset_index(drop=True,inplace=True)

In [0]:
face_result = df["FACIAL_result"] == "clear"
docs_result = df["DOCS_result"] == "clear"
df['Pass'] = face_result & docs_result

In [0]:
one_attempt_users = df.drop_duplicates('user_id', False)

In [0]:
multiple_user_attempts = df[df.duplicated('user_id', keep=False)]
df_multiple_1st = multiple_user_attempts.drop_duplicates('user_id', 'first')
df_multiple_last = multiple_user_attempts.drop_duplicates('user_id', 'last') 
df_multiple_passed1st = df_multiple_1st[df_multiple_1st["Pass"]]

In [0]:
df_multiple_failed_first = df_multiple_1st[df_multiple_1st['Pass'] == False]
df_2nd_attempt = multiple_user_attempts.groupby(['user_id']).head(2).groupby(['user_id']).tail(1)
df_first_attempt = df.drop_duplicates('user_id', 'first')

In [0]:
x1 = pd.concat([df_multiple_failed_first, df_2nd_attempt])
x1 = x1.drop_duplicates('user_id', 'last')
y2 = pd.concat([df_first_attempt, x1])
y2 = y2.drop_duplicates('user_id', 'last')
df_pass_rate = y2.sort_values(["created_at"])
df_pass_rate.reset_index(drop=True, inplace=True)

In [41]:
df.head()

,user_id,FACIAL_result,face_comparison_result,created_at,facial_image_integrity_result,visual_authenticity_result,properties,FACIAL_attempt_id,DOCS_result,image_integrity_result,...,sub_result,gender,nationality,document_type,date_of_expiry,issuing_country,issuing_date,issuing_state,document_version,Pass
0,eb880365e8224b469d4503e3434b2853,clear,clear,2017-05-23,clear,clear,{'score': 0.66},3f43101902d247eeb7d7cf5720328b58,clear,clear,...,clear,Female,CHN,passport,2025-09-06,CHN,NaN,NaN,NaN,True
1,c7d536122c2c494cb08757c6e2806093,clear,clear,2017-05-23,clear,clear,{'score': 0.69},002498134c1b47c987277a3ada7f06dc,clear,clear,...,clear,Male,POL,national_identity_card,2025-12-28,POL,NaN,NaN,NaN,True
2,6e01ed0df18948808d4d8b2762ccd116,clear,clear,2017-05-23,clear,clear,{'score': 0.69},71c31b8451424a78a5899140b08dfca8,clear,clear,...,clear,NaN,NaN,driving_licence,2031-06-10,GRC,NaN,NaN,NaN,True
3,6e01ed0df18948808d4d8b2762ccd116,clear,clear,2017-05-23,clear,clear,{'score': 0.69},85f99d1bd1ee4ece9b5eef157893b3c6,clear,clear,...,clear,NaN,NaN,driving_licence,2031-06-10,GRC,NaN,NaN,NaN,True
4,4b939fe40b2346c5a8b820fa9977e1ef,clear,clear,2017-05-23,clear,clear,{'score': 0.52},88139856c9764fb1854be54a0de8794f,clear,clear,...,clear,Male,NaN,driving_licence,2022-07-30,GBR,NaN,NaN,NaN,True


In [42]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [0]:
df.to_csv('data_cleansed.csv')
!cp data_cleansed.csv drive/My\ Drive/